# 1. Read articles and get idea of what to do
2. Dataset -> pre process and clean up. Remove stop words, lemmatize, 
3. Develop my own method to extract aspect terms.
4. evaluate, compare to other studies


In [1]:
# !pip install spacy==2.1.0

In [2]:
# !python3 -m spacy download en_core_web_lg

In [3]:
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

In [4]:
# !pip install neuralcoref --no-binary neuralcoref

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from collections import namedtuple, OrderedDict

In [7]:
# dataset = xml.dom.minidom.parse("test_datasets/restaurants-trial.xml")
root = ET.parse("test_datasets/restaurants-trial.xml").getroot()
dataset = ET.parse("test_datasets/restaurants-trial.xml").getroot().findall('sentence')

actual_dataset = ET.parse("test_datasets/restaurants-trial.xml").getroot().findall('sentence')

In [8]:
# print(dataset.firstChild.tagName)
print(len(dataset))

100


In [9]:
class Aspect:
    '''Aspect objects contain the term (e.g., battery life) and polarity (i.e., pos, neg, neu, conflict) of an aspect.'''

    def __init__(self, term, polarity, offsets):
        self.term = term
        self.polarity = polarity
        self.offsets = offsets

    def create(self, element):
        self.term = element.attrib['term']
        self.polarity = element.attrib['polarity']
        self.offsets = {'from': str(element.attrib['from']), 'to': str(element.attrib['to'])}
        return self

    def update(self, term='', polarity=''):
        self.term = term
        self.polarity = polarity

In [10]:
def validate(filename):
    elements = filename
    aspects = []
    text = []
    for e in elements:
        sent_id = e.attrib['id']
        for f in e.findall('text'):
            text.append((f, sent_id))
        for eterms in e.findall('aspectTerms'):
            if eterms is not None:
                for a in eterms.findall('aspectTerm'):
                    aspects.append((Aspect('', '', []).create(a), sent_id))
    return text, aspects

In [11]:
text = validate(dataset)[0]
print(len(text))
sent_id = text[0][1]
print("id: ", sent_id)

print (text[1][0].text)

aspects = validate(dataset)[1]
for i in range(len(aspects)):
    if(aspects[i][1] == sent_id):
        print(aspects[i][0].term)


100
id:  813
And really large portions.
appetizers
salads
steak
pasta


In [12]:
for sentence in root.findall("./sentence/[@id = '813']"):
    print (sentence.find("text").text)

All the appetizers and salads were fabulous, the steak was mouth watering and the pasta was delicious!!!


In [13]:
#LEXICON - subjectivity clues Bing Liu
SubClue = namedtuple('SubClue', ['rel', 'pri_pol', 'stemmed'])
class SubjectivityClues(object):
    # map POS tags in subjectivty clues to spacy tags:
    _pos_map = {
        'noun': 'NOUN',
        'verb': 'VERB',
        'adj': 'ADJ',
        'adverb': 'ADV',
        'anypos': 'ANYPOS',
        }

    def __init__(self, sc_path=
            'lexicons/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'):

        # load subjectivity clues
        with open(sc_path, 'r') as fo:
            sub_clues = [c.strip().replace('type=', 'rel=').split()
                         for c in fo]
        sub_clues = [dict([d.split('=') for d in sc if len(d.split('=')) == 2])
                     for sc in sub_clues]

        # convert to dict keyed by (word, part-of-speech) and exclude neutral
        # polarity clues:
        # ISSUE: if stemmed for matches non-stemmed will overwrite (may not
        # have impact as polarity and type remain the same
        self.lex = {(s['word1'], self._pos_map[s['pos1']]): SubClue(s['rel'],
                     s['priorpolarity'],
                     True if s['stemmed1'] == 'y' else False)
                    for s in sub_clues if s['priorpolarity']}
        # sclues_pos and sclues_any are just sets containing all (word, pos)
        # tuples with a proper POS tag and 'ANYPOS' respectively
        self.sclues_pos = set([(w, t) for w, t in self.lex.keys()
                               if t != 'ANYPOS'])
        self.sclues_any = set(self.lex.keys()).difference(self.sclues_pos)

    def lookup(self, token):
        """
        Returns a `SubClue` named tuple for `token` if either `(token.norm_,
        token.pos_)` or `(token.norm_, 'ANYPOS') is in self.lex. Returns an
        empty tuple if no match is found.
        """
        tnp = (token.norm_, token.pos_)
        tna = (token.norm_, 'ANYPOS')
        if tnp in self.lex or tna in self.lex:
            key = tnp if tnp in self.lex else tna
            return self.lex[key]
        else:
            return tuple()

In [14]:
subjclues = SubjectivityClues()

In [15]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
# from spacy.util import filter_spans
from spacy.lang.en.stop_words import STOP_WORDS
import string
import neuralcoref

punc = string.punctuation
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_lg')
pronoun_list=['he','she','it','they','them','him','her','his','hers','its','we','us']

In [16]:
neuralcoref.add_to_pipe(nlp)

In [17]:
#get verb chunks
def get_verb_chunks(doc):
    matcher = Matcher(nlp.vocab)
    
    pattern = [{'POS':'VERB', "OP":"*"},
              {'POS':'ADV', "OP":"*"},
              {'POS':'PART', "OP":"*"},
              {'POS':'VERB', "OP":"+"},
              {'POS':'PART', "OP":"*"}]
    
    matcher.add("verb_clause", None, pattern)
    matches = matcher(doc)
    k = len(matches) - 1
    
    span = doc[matches[k][1]:matches[k][2]]
    
    return(span)

In [18]:
doc = nlp("The food is great and reasonably priced.")
for chunk in get_verb_chunks(doc):
    print (chunk.text)

priced


In [19]:
# noun_chunks from spacy

from spacy.symbols import NOUN, PROPN, PRON

def noun_chunks(obj):
    """
    Detect base noun phrases from a dependency parse. Works on both Doc and Span.
    """
    labels = [
        "nsubj",
        "dobj",
        "nsubjpass",
        "pcomp",
        "pobj",
        "dative",
        "appos",
        "attr",
        "ROOT",
    ]
    doc = obj.doc  # Ensure works on both Doc and Span.
    np_deps = [doc.vocab.strings.add(label) for label in labels]
    conj = doc.vocab.strings.add("conj")
    np_label = doc.vocab.strings.add("NP")
    seen = set()
    for i, word in enumerate(obj):
#         print("i", i)
#         print("word", word)
        if word.pos not in (NOUN, PROPN, PRON):
            continue
        # Prevent nested chunks from being produced
        if word.i in seen:
            continue
        if word.dep in np_deps:
            if any(w.i in seen for w in word.subtree):
                continue
            seen.update(j for j in range(word.left_edge.i, word.i + 1))
            yield word.left_edge.i, word.i + 1, np_label
        elif word.dep == conj:
            head = word.head
            while head.dep == conj and head.head.i < head.i:
                head = head.head
            # If the head is an NP, and we're coordinated to it, we're an NP
            if head.dep in np_deps:
                if any(w.i in seen for w in word.subtree):
                    continue
                seen.update(j for j in range(word.left_edge.i, word.i + 1))
                yield word.left_edge.i, word.i + 1, np_label


SYNTAX_ITERATORS = {"noun_chunks": noun_chunks}

In [20]:
doc = nlp("Our agreed favorite is the orrechiete with sausage and chicken (usually the waiters are kind enough to split the dish in half so you get to sample both meats).")

# for chunk in doc.noun_chunks:
#     print(chunk.text)

for chunk in noun_chunks(doc):
    if(chunk[1] - chunk[0] == 1):
        print(doc[chunk[0]])
    else:
        for i in range(chunk[1]-chunk[0]):
            print(doc[chunk[0]+i], end = " ")
        print()


Our agreed favorite 
the orrechiete 
sausage
chicken
the waiters 
the dish 
half
you
both meats 


In [21]:
doc = nlp("Even though its good seafood, the prices are too high.")
for token in doc:
    if(subjclues.lookup(token)):
        print(token.text, '\t', token.pos_, '\t', subjclues.lookup(token))

Even 	 ADV 	 SubClue(rel='weaksubj', pri_pol='positive', stemmed=False)
though 	 ADP 	 SubClue(rel='strongsubj', pri_pol='neutral', stemmed=False)
good 	 ADJ 	 SubClue(rel='weaksubj', pri_pol='positive', stemmed=False)
too 	 ADV 	 SubClue(rel='weaksubj', pri_pol='negative', stemmed=True)
high 	 ADJ 	 SubClue(rel='weaksubj', pri_pol='neutral', stemmed=False)


In [22]:
new_stopwords = ['time', 'trouble', 'notch', 'the', 'ie', 'eg', 'an', 'thing', 'mouth', 'restaurant', 'bit']
for word in new_stopwords:
    stopwords.append(word)
print(stopwords)

['last', 'hereafter', 'get', 'due', 'whereby', 'give', 'with', "'s", 'besides', 'might', 'amount', 'thereafter', 'was', 'part', 'fifteen', 'in', 'side', 'latterly', 'one', 'less', 'any', 'every', 'please', 'regarding', 'keep', 'though', 'across', 'it', 'here', 'becoming', 'together', 'take', 'before', 'to', 'call', 'and', 'us', 'eight', 'everything', 'someone', 'her', 'will', 'somehow', 'two', 'via', 'may', 'cannot', 'do', 'least', 'back', 'can', 'too', "'re", 'at', 'still', 'using', 'former', 'an', 'himself', 'made', 'thence', 'another', 'enough', 'only', 'whereas', 'did', 'whom', 'quite', 'indeed', 'during', 'beyond', 'thereby', 'toward', 'its', 'which', 'hence', 'next', 'because', 'namely', 'you', 'is', 'afterwards', 'but', 'front', 'doing', 'wherein', 'well', 'am', 'the', 'moreover', 'yourselves', 'what', 'through', 'whether', 'twelve', 'noone', 'has', 'sometimes', 'out', 'either', 'nobody', 'where', 'perhaps', 'there', 'myself', 'therefore', 'without', 'below', 'bottom', 'anyway',

In [23]:
more_subj = ['deficiency', 'consistently', 'mouth watering', 'all costs']

In [24]:
def pre_process(sent):
    doc = nlp(sent)
    
    lemmad = ""
    
    #lemmatization
    for token in doc:
        if token.text not in stopwords:
            if token.lemma_ != "-PRON-":
                temp = token.lemma_.lower().strip()
            else:
                temp = token.lower_
            lemmad = lemmad + temp + " "
    lemmad = nlp(lemmad)
    
    #remove stop words and punctuation
    cleaned_tokens = ""
    for token in lemmad:
        if token.text not in stopwords and token.text not in punc:
            cleaned_tokens = cleaned_tokens + token.text + " "
    
    cleaned_tokens = nlp(cleaned_tokens)
    return cleaned_tokens

In [25]:
sent = "Even though its good seafood, the prices are too high."
raw_doc = nlp(sent)
print(raw_doc)

#pre processed
pp_doc = pre_process(sent)
print(pp_doc)

Even though its good seafood, the prices are too high.
good seafood price high 


In [26]:
for token in raw_doc:
    print(token.text, '\t', token.dep_, '\t', token.pos_)

Even 	 advmod 	 ADV
though 	 mark 	 ADP
its 	 poss 	 DET
good 	 amod 	 ADJ
seafood 	 nsubj 	 NOUN
, 	 punct 	 PUNCT
the 	 det 	 DET
prices 	 nsubj 	 NOUN
are 	 ROOT 	 VERB
too 	 advmod 	 ADV
high 	 acomp 	 ADJ
. 	 punct 	 PUNCT


In [27]:
ents_list = []
for ent in raw_doc.ents:
    ents_list.append(ent.text)
    
for chunk in raw_doc.noun_chunks:
    print(chunk)
    
    if chunk.text in ents_list:
        print("here")
            
print("/////")

for ent in raw_doc.ents:
    print(ent)

its good seafood
the prices
/////


In [28]:
# printing original token of those in the pre processed document.
for token in raw_doc:
    if token.lemma_.lower() in pp_doc.text and token.lemma_.lower() != "the":
        if(subjclues.lookup(token) or token.lemma_.lower() in more_subj):
            print(token.text, '\t', token.pos_, '\t', subjclues.lookup(token))
        
        lefts = [t.text for t in token.lefts]
        rights = [t.text for t in token.rights]
#         print("lefts", lefts)
#         print("rights", rights)

        printed = False
    
        if(lefts and rights):
            if(lefts[0] == '(' and rights[0] == ')'):
                print(lefts[0]+token.text+rights[0], '\t', token.dep_, '\t', token.pos_)
                printed = True
        if not printed:
            print(token.text, '\t', token.dep_, '\t', token.pos_)

good 	 ADJ 	 SubClue(rel='weaksubj', pri_pol='positive', stemmed=False)
good 	 amod 	 ADJ
seafood 	 nsubj 	 NOUN
prices 	 nsubj 	 NOUN
high 	 ADJ 	 SubClue(rel='weaksubj', pri_pol='neutral', stemmed=False)
high 	 acomp 	 ADJ


In [29]:
#EXTRACT ASPECTS
my_aspects = []
token_aspects = []
for i in range(len(text)):
    current_sent = text[i][0].text
    current_id = text[i][1]
    raw_doc = nlp(current_sent)
    pp_doc = pre_process(current_sent)
    temp_aspect = ""
    
#     list of ents
    ents_list = []
    for ent in raw_doc.ents:
        ents_list.append(ent.text)
    
    coref_clusters = raw_doc._.coref_clusters

    for chunk in raw_doc.noun_chunks:
        #do this at start in case next noun chunk is part of the previous one e.g. okra (bindi)
        if (chunk.text[0] == "("):
            temp_aspect = temp_aspect + chunk.text
            temp_aspect += ") "
            previous_i = current_i
            current_i = token.i
            chunk_span = raw_doc[previous_i:current_i+1]
            continue
        
        if(temp_aspect != ''):
            temp_aspect = temp_aspect[:-1]
            my_aspects.append((temp_aspect, current_id))
            token_aspects.append((chunk_span, current_id))
        temp_aspect = ""
        
        if chunk.text in ents_list:
            continue
        
        for cluster in coref_clusters:
            if chunk in cluster and chunk != cluster[0]:
                continue

        #find index of first key aspect word
        for token in chunk:
            if token.lemma_.lower() in pp_doc.text and not subjclues.lookup(token) and token.lemma_.lower() not in more_subj:
                #remove useless words 'the', 'a', etc before a noun. 
                if((token.lemma_.lower() == "the" or token.lemma_.lower() == "a") and token.dep_ == "det"):
                    continue
#                 print(token.text)
                current_i = token.i
                break
                
        for token in chunk:
            #add if its in the pre-processsed doc (meaning its important) and NOT in lexicon.
            if token.lemma_.lower() in pp_doc.text and not subjclues.lookup(token) and token.lemma_.lower() not in more_subj:
                #remove useless words 'the', 'a', etc before a noun. 
                if((token.lemma_.lower() == "the" or token.lemma_.lower() == "a") and token.dep_ == "det"):
                    continue
                if(token.text in ents_list):
                    continue
#                 print(token.text)
                previous_i = current_i
                current_i = token.i
                temp_aspect = temp_aspect + token.text + " "
                
                chunk_span = raw_doc[previous_i:current_i+1]
#         print("span",chunk_span)
#         print("aspect",temp_aspect)        
    #last time to get the last aspect
    if(temp_aspect != ''):
            temp_aspect = temp_aspect[:-1]
            my_aspects.append((temp_aspect, current_id))
            token_aspects.append((chunk_span, current_id))
      

my_aspects

[('appetizers', '813'),
 ('salads', '813'),
 ('steak', '813'),
 ('watering', '813'),
 ('pasta', '813'),
 ('portions', '1579'),
 ('lassi', '2882'),
 ('lamb chettinad', '2882'),
 ('garlic naan', '2882'),
 ('rasamalai', '2882'),
 ('Service', '1609'),
 ('bills', '3018'),
 ('Service', '3292'),
 ('seafood', '3440'),
 ('prices', '3440'),
 ('addition', '870'),
 ('food', '870'),
 ('prices', '870'),
 ('food', '2507'),
 ('garlic shrimp', '3081'),
 ('okra', '3081'),
 ('bindi', '3081'),
 ('lamb', '3081'),
 ('menu', '1884'),
 ('selections', '1884'),
 ('burger', '1884'),
 ('breads', '2077'),
 ('food', '201'),
 ('price', '201'),
 ('wine list', '201'),
 ('food', '3049'),
 ('portions', '3049'),
 ('coconut rice', '1242'),
 ('Tom Kha soup', '2211'),
 ('food', '3343'),
 ('crunchy tuna', '2141'),
 ('prices', '83'),
 ('Sala', '381'),
 ('service', '3315'),
 ('food', '3315'),
 ('price', '3315'),
 ('beginning', '3315'),
 ('food', '1709'),
 ('food', '193'),
 ('reasons', '2459'),
 ('calzones', '1293'),
 ('tips', 

In [30]:
for current_token in token_aspects:
    max_sim = 0
    sum_sim = 0
    avg_sim = 0
    for compare_token in token_aspects:
        if current_token[1] != compare_token[1]:
            if(current_token[0].similarity(compare_token[0])):
                sim_score = current_token[0].similarity(compare_token[0])
                sum_sim += sim_score
                if max_sim < sim_score:
                    max_sim = sim_score
#     print(current_token[0], avg_sim, max_sim)
    avg_sim = sum_sim / len(token_aspects)
    if (avg_sim < 0.17 or max_sim < 0.60) and avg_sim != 0.0:
#     if avg_sim < 0.30 and max_sim < 0.53:
        if((current_token[0].text, current_token[1]) in my_aspects):
            my_aspects.remove((current_token[0].text, current_token[1]))
            print(current_token[0], avg_sim, max_sim)
            print("\tremoved")

watering 0.25204514413618084 0.446450245195195
	removed
lassi 0.0946155061908893 0.4269146542649121
	removed
bills 0.18739197509903072 0.573517877350986
	removed
addition 0.2438230634555494 0.4986083795451099
	removed
bindi -0.0006434869015635305 0.31746877948368774
	removed
menu 0.28032961376081605 0.5310790559565972
	removed
Sala 0.044741066006610014 0.2611578706321381
	removed
beginning 0.20776557813534097 0.5355942979744079
	removed
reasons 0.23896812079279373 0.596764846284457
	removed
calzones 0.12648731817442488 0.5383531232314991
	removed
tips 0.20676653576281945 0.4424892874515434
	removed
woman 0.1975070254490384 0.5618484262173992
	removed
spreads 0.15812113426369453 0.3035282482031155
	removed
Ruby Foo 0.13982881779427336 0.4080430243701792
	removed
work 0.23067486292963418 0.5661829527584181
	removed
group 0.15871121658826423 0.4163860281580687
	removed
boyfriend 0.18269706751717343 0.5278435031266772
	removed
birthday 0.2112237437685425 0.5475895588717791
	removed
person 

In [31]:
my_aspects

[('appetizers', '813'),
 ('salads', '813'),
 ('steak', '813'),
 ('pasta', '813'),
 ('portions', '1579'),
 ('lamb chettinad', '2882'),
 ('garlic naan', '2882'),
 ('rasamalai', '2882'),
 ('Service', '1609'),
 ('Service', '3292'),
 ('seafood', '3440'),
 ('prices', '3440'),
 ('food', '870'),
 ('prices', '870'),
 ('food', '2507'),
 ('garlic shrimp', '3081'),
 ('okra', '3081'),
 ('lamb', '3081'),
 ('selections', '1884'),
 ('burger', '1884'),
 ('breads', '2077'),
 ('food', '201'),
 ('price', '201'),
 ('wine list', '201'),
 ('food', '3049'),
 ('portions', '3049'),
 ('coconut rice', '1242'),
 ('Tom Kha soup', '2211'),
 ('food', '3343'),
 ('crunchy tuna', '2141'),
 ('prices', '83'),
 ('service', '3315'),
 ('food', '3315'),
 ('price', '3315'),
 ('food', '1709'),
 ('food', '193'),
 ('turnip cake', '2105'),
 ('bagels', '2152'),
 ('ambiance', '882'),
 ('atmosphere', '882'),
 ('food', '882'),
 ('service', '882'),
 ('waitress', '1124'),
 ('curry row', '3546'),
 ('fish', '3020'),
 ('rice', '3020'),
 ('

In [32]:
#the true aspects for each sentence
true_aspects = []
for i in range(len(aspects)):
    true_aspects.append((aspects[i][0].term, aspects[i][1]))
true_aspects

[('appetizers', '813'),
 ('salads', '813'),
 ('steak', '813'),
 ('pasta', '813'),
 ('portions', '1579'),
 ('sweet lassi', '2882'),
 ('lamb chettinad', '2882'),
 ('garlic naan', '2882'),
 ('rasamalai', '2882'),
 ('Service', '1609'),
 ('bills', '3018'),
 ('Service', '3292'),
 ('lambc hops', '2041'),
 ('seafood', '3440'),
 ('prices', '3440'),
 ('food', '870'),
 ('prices', '870'),
 ('food', '2507'),
 ('garlic shrimp', '3081'),
 ('okra (bindi)', '3081'),
 ('lamb', '3081'),
 ('menu', '1884'),
 ('burger', '1884'),
 ('steak', '1884'),
 ('escargot', '1884'),
 ('breads', '2077'),
 ('food', '201'),
 ('price', '201'),
 ('wine list', '201'),
 ('food', '3049'),
 ('portions', '3049'),
 ('coconut rice', '1242'),
 ('Tom Kha soup', '2211'),
 ('food', '3343'),
 ('crunchy tuna', '2141'),
 ('prices', '83'),
 ('service', '3315'),
 ('food', '3315'),
 ('price', '3315'),
 ('food', '1709'),
 ('food', '193'),
 ('priced', '193'),
 ('calzones', '1293'),
 ('turnip cake', '2105'),
 ('roast pork buns', '2105'),
 ('eg

In [33]:
#compare
total_true = len(true_aspects)
total_returned = len(my_aspects)

correct = 0
for aspect in my_aspects:
    if aspect in true_aspects:
        correct = correct + 1
#     else:
#         correct -= 1

# percent = (correct / total_true) * 100
# print(percent, "%")

precision = correct /total_returned 
print("precision ", precision)

recall = correct / total_true
print("recall", recall)

f_score = 2 * (precision * recall) / (precision + recall)
print("f_score", f_score)

precision  0.7872340425531915
recall 0.7708333333333334
f_score 0.7789473684210526


In [34]:
#refine rule based method
#can never achieve full coverage with rule based
#not generalised method - cant continually add rules
#is there any way to generalise 
#look at papers, research new ways to combine with rule based

In [35]:
len(my_aspects)

94

In [36]:
len(true_aspects)    

96

In [37]:
with open('test.txt', 'a') as f:
    for i in range(len(my_aspects)):
        print(my_aspects[i], file = f)

In [38]:
doc1 = nlp("Scientists know many things about the Sun. They know how old it is. The Sun is more than 4½ billion years old. It is also a star that is the centre of our solar system. They also know the Sun’s size.")


In [39]:
coref_clusters = doc1._.coref_clusters
print(coref_clusters[0])
# print(coref_clusters[0][1])

print(coref_clusters[1])
# print(coref_clusters[1][2])

coref_clusters[1].mentions[0]



Scientists: [Scientists, They, They]
the Sun: [the Sun, The Sun, Sun, It, Sun]


the Sun

In [40]:
pronoun_list=['he','she','it','they','them','him','her','his','hers','its','we','us']
# for chunk in doc1.noun_chunks:
#     print(chunk)

# print()
# print()
# print()
for chunk in doc1.noun_chunks:
    for cluster in coref_clusters:
#         print(cluster)
#         print(cur_span)
        if chunk in cluster:
            print(chunk)

#             if(chunk.text.lower() in pronoun_list):
#                 print("\tremoved")
            
            if(chunk != cluster[0]):
                print('\tremoved')

Scientists
the Sun
They
	removed
	removed
The Sun
	removed
It
	removed
	removed
They
	removed
	removed
